In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression as Lin_Reg

from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import Lasso as Lasso_Reg
from statsmodels.regression.linear_model import OLS
import sklearn.preprocessing as Preprocessing

from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso

import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression
%matplotlib inline

import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
from itertools import combinations
%matplotlib inline

In [ ]:
df = pd.read_csv('/content/kc_house_data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

there are 21613 samples with 21 features initially

feature description: id : A notation for a house

date: Date house was sold

price: Price is prediction target

bedrooms: Number of bedrooms

bathrooms: Number of bathrooms

sqft_living: Square footage of the home

sqft_lot: Square footage of the lot

floors :Total floors (levels) in house

waterfront :House which has a view to a waterfront

view: Has been viewed

condition :How good the condition is overall

grade: overall grade given to the housing unit, based on King County grading system

sqft_above : Square footage of house apart from basement

sqft_basement: Square footage of the basement

yr_built : Built Year

yr_renovated : Year when house was renovated

zipcode: Zip code

lat: Latitude coordinate

long: Longitude coordinate

sqft_living15 : Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area

sqft_lot15 : LotSize area in 2015(implies-- some renovations)

# EDA & preprocessing

In [ ]:
df.isnull().sum()
# no missing value

In [ ]:
df['floors'].value_counts().to_frame()

In [ ]:
sns.boxplot(df['waterfront'], df['price'])
plt.title("'Boxplot 'Waterfront' vs 'Price'")

In [ ]:
sns.regplot(df['sqft_above'], df['price'])
plt.title("Regression Plot")
print("We can see that it is positively correlated.")

In [ ]:
df.corr()['price'].sort_values()

In [ ]:
cor = df.corr()
plt.figure(figsize=(20,15))
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].apply(lambda date:date.month)
df['year'] = df['date'].apply(lambda date:date.year)

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
df.groupby('month').mean()['price'].plot()

plt.subplot(2, 2, 2)
df.groupby('year').mean()['price'].plot()

In [ ]:
df = df.drop('date',axis=1)
df = df.drop('id',axis=1)

In [ ]:
df.bathrooms.unique()

In [ ]:
plt.hist(df.bathrooms,rwidth = 0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("count")

In [ ]:
sns.distplot(df['price'])

In [ ]:
# monthly change of prices
df['ym'] = (df.index.year *100 + df.index.month).astype(str) 
ym_summary = df.groupby('ym')['price'].agg(['mean','count'])

vmin = np.min(ym_summary['count'])
vmax = np.max(ym_summary['count'])
norm = colors.Normalize(vmin,vmax)

plt.figure(figsize=(15,7))
plt.scatter(x=np.arange(ym_summary.shape[0]), y =ym_summary['mean'],c= ym_summary['count'],
            s= ym_summary['count'],norm=norm ,alpha = 0.8, cmap='jet')

plt.plot(np.arange(ym_summary.shape[0]), ym_summary['mean'] ,'--')
plt.xticks(np.arange(ym_summary.shape[0]),ym_summary.index.values)
plt.yscale('log')
plt.xlabel('Year-Month')
plt.ylabel('Price (log scale)')
clb = plt.colorbar() 
clb.ax.set_title('number of sales')
plt.title('Averge Price by Month')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
vmin = np.min(df.price)
vmax = np.max(df.price)
norm = colors.LogNorm(vmin*2,vmax/3)
plt.scatter(df.long,df.lat, marker='*',c=df.price,norm=norm,cmap='jet') 
plt.xlabel('Longitude')
plt.ylabel('Latituede')
plt.title('House Price by Geography')
clb = plt.colorbar() 
clb.ax.set_title('Price')

In [ ]:
def scatter_p(fea,price):
    n_f = len(fea)
    n_row = n_f//3+1
    fig=plt.figure(figsize=(20,15))
    i = 1
    for f in fea:
        x=df[f]
        y=df[price]
        m, b = np.polyfit(x, y, 1)    
        
        ax=fig.add_subplot(n_row,3,i)
        plt.plot(x,y,'.',color='b')
        plt.plot(x, m*x + b, '-',color='r')
        plt.xlabel(f)
        plt.ylabel(price)
        i += 1

In [ ]:
scatter_p(fea=['sqft_living', 'sqft_lot','sqft_basement'],price='price')

In [ ]:
df['log_sqft_living'] = np.log(df['sqft_living'])
df['log_sqft_lot'] = np.log(df['sqft_lot'])
df['log_sqft_basement'] = np.log1p(df['sqft_basement'])
scatter_p(fea=['log_sqft_living','log_sqft_lot','log_sqft_basement'],price='log_price')

In [ ]:
df['basement_ind'] = [1 if x>0 else 0 for x in df.sqft_basement]
scatter_p(fea=['yr_built','yr_renovated','bathrooms'],price='log_price')

In [ ]:
df['renovated_ind'] = [1 if x>0 else 0 for x in df.yr_renovated]
x=df.loc[df.renovated_ind==1,'yr_renovated']
y=df.loc[df.renovated_ind==1 ,'log_price']
m, b = np.polyfit(x, y, 1)   
plt.plot(x,y,'.',color='b')
plt.plot(x, m*x + b, '-',color='r')
plt.title('Renovated Houses')
plt.xlabel('yr_renovated')
plt.ylabel('log_price')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,12))
fig.add_subplot(3,2,1)
sns.violinplot(x="bedrooms", y="log_price", data=df)    
fig.add_subplot(3,2,2)
sns.violinplot(x="condition", y="log_price", data=df)  
fig.add_subplot(3,2,3)
sns.violinplot(x="grade", y="log_price", data=df)   
fig.add_subplot(3,2,4)
sns.violinplot(x="view", y="log_price", data=df)  
fig.add_subplot(3,2,5)
sns.violinplot(x="floors", y="log_price", data=df)  

In [ ]:
# convert bedrooms, floors, year_month to binary feature
data = pd.get_dummies(df,columns=['bedrooms','floors','ym'],drop_first=True)

In [ ]:
for x in ['price']:
    q75,q25 = np.percentile(df.loc[:,x],[75,25])
    intr_qr = q75-q25
 
    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)
 
    df.loc[df[x] < min,x] = np.nan
    df.loc[df[x] > max,x] = np.nan

In [ ]:
df = df.dropna()

# interactive map

In [ ]:

  import folium
  import matplotlib.cm

In [ ]:
# define color vector function
def ColorVector(values, number_of_colors, cmap_name):
    buckets = pd.qcut(values, number_of_colors).codes
    cmap = matplotlib.cm.get_cmap(name = cmap_name)
    colors = []
    for i in range(0, number_of_colors):
        r = int(cmap(i/number_of_colors)[0]*255)
        g = int(cmap(i/number_of_colors)[1]*255)
        b = int(cmap(i/number_of_colors)[2]*255)
        color = "#{0:02x}{1:02x}{2:02x}".format(r, g, b)  
        colors.append(color)
    values = []
    for j in range(0, len(buckets)):
        value = colors[buckets[j]]
        values.append(value)
    return values
  
# define color categories function
def ColorCategories(variable, colors):
    properties['Colors'] = colors
    counts = properties.groupby(['Colors', variable]).size().reset_index().rename(columns={0:'count'})
    color_list = counts.Colors.unique()
    for color in color_list:
        temp = counts[(counts.Colors == color)]
        print(color, min(temp[variable]), max(temp[variable]), len(temp))
        
# read in the data
properties = pd.read_csv('/content/kc_house_data.csv')

# prepare map components
lats = properties['lat']
lngs = properties['long']
vals = properties['price']
addresses = properties['zipcode']
sqft = properties['sqft_living']
# owners = properties['Owner']

##### View 1 - Year Built #####

#years = []
#for year in year_built:
#    if pd.isnull(year) == False:
#        year = int(year)
#    else:
#        year = None
#    years.append(year)
    
# coloring    
#properties['YearBuilt'] = years
years = properties['yr_built'] 
colors = ColorVector(properties['yr_built'].values, 20, 'coolwarm')
ColorCategories('yr_built', colors)

# determine color vector
colors = ColorVector(properties['yr_built'].values, 20, 'coolwarm')

# create base map
m = folium.Map(location=[np.mean(properties.lat), np.mean(properties.long)], 
               tiles = 'Stamen Toner', zoom_start = 13)

# loop through properties to add map layers
for i in range(len(lats)):
    folium.Circle(location=[lats[i], lngs[i]], 
                  popup = ("""<b>{}</b><br>Year Built: {}""".format(addresses[i], years[i])),
                  radius = 20, 
                  color = colors[i],
                  fill = True,
                  fill_color = colors[i], 
                  fill_opacity=.30).add_to(m)
    
# save map as html file    
m.save('view-1.html')

this view shows clusters of homes built in similar time periods and paints a picture of development over time.

the color spectrum plots blue for older houses and red for newer houses.

In [ ]:
# stylize as currency
dollar_vals = []
for i in range(0, len(vals)):
    dollar_vals = '${:0,.0f}'.format(vals[i]).replace('$-','-$')
    
# stylize as number (commas)
num_sqfts = []
for i in range(0, len(sqft)):
    num_sqft = '{:0,.0f}'.format(sqft[i])
    num_sqfts.append(num_sqft)
    
# determine color vector    
colors = ColorVector(properties['price'].values, 10, 'plasma')
ColorCategories('price', colors)

# create the base map
m = folium.Map(location=[np.mean(properties.lat), np.mean(properties.long)], 
               tiles = 'Stamen Toner', zoom_start = 13)

# loop through properties to add map layers
for i in range(len(lats)):
    folium.Circle(location=[lats[i], lngs[i]], 
                  popup = ("""<b>{}</b><br>{}<br>{} sq. ft.<br>""".format(addresses[i], vals[i], sqft[i])),
                  radius = 20, 
                  color = colors[i],
                  fill = True,
                  fill_color = colors[i], 
                  fill_opacity=.30).add_to(m)
# save map as html file      
m.save('view-2.html')

In this heatmap, the brighter the dot the higher the assessed value. Clicking on a circle reveals the total assessed value for the current tax year as well as the square footage of the home.

# Predicting Price

## simple linear regression

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
print("done")

In [ ]:
# simple linear regression to explore features we think are interesting

#Define target 'y'(dependendent variable) and features 'X'(independent variables or predictors).
import statsmodels.formula.api as smf
y_2 = df['price']
X_2 = df.drop('price',axis=1)
#Test each continus variable as a standalone linear regression.
col_names = X_2.select_dtypes('float').columns
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Selling price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = smf.ols(formula=f, data=df).fit()
    X_new = pd.DataFrame({val: [X_2[val].min(), X_2[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[idx+1])

In [ ]:
#Define target 'y'(dependendent variable) and features 'X'(independent variables or predictors).
import statsmodels.formula.api as smf
y_2 = df['price']
X_2 = df.drop('price',axis=1)
#Test each continus variable as a standalone linear regression.
col_names = X_2.select_dtypes('int').columns
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Selling price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = smf.ols(formula=f, data=df).fit()
    X_new = pd.DataFrame({val: [X_2[val].min(), X_2[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[idx+1])

In [ ]:
cols_yreno = [col for col in X_2.columns if 'yr_renovated' in col]
cols_zipcode = [col for col in X_2.columns if 'zipcode' in col]
cols_grade = [col for col in X_2.columns if 'grade' in col]
cols=[cols_yreno,cols_zipcode,cols_grade]

for col in cols:
    sum_cols = "+".join(col)
    f = "price ~" + sum_cols
    model = smf.ols(formula= f, data= df).fit()
    print(model.summary())

## RFE

In [ ]:
# Define dictionary to store our rankings
ranks = {}
# Create our function which stores the feature rankings to the ranks dictionary
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [ ]:
colnames = X.columns

In [ ]:
# Construct our Linear Regression model
lr = LinearRegression(normalize=True)
lr.fit(X,y)
#stop the search when only the last feature is left
rfe = RFE(lr, n_features_to_select=1, verbose =3 )
rfe.fit(X,y)
ranks["RFE"] = ranking(list(map(float, rfe.ranking_)), colnames, order=-1)

In [ ]:
# Using Linear Regression
lr = LinearRegression(normalize=True)
lr.fit(X,y)
ranks["LinReg"] = ranking(np.abs(lr.coef_), colnames)

# Using Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X,y)
ranks['Ridge'] = ranking(np.abs(ridge.coef_), colnames)

# Using Lasso
lasso = Lasso(alpha=.05)
lasso.fit(X, y)
ranks["Lasso"] = ranking(np.abs(lasso.coef_), colnames)

rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=3)
rf.fit(X,y)
ranks["RF"] = ranking(rf.feature_importances_, colnames)

In [ ]:
r = {}
for name in colnames:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
 
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("\t%s" % "\t".join(methods))
for name in colnames:
    print("%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods]))))

In [ ]:
meanplot = pd.DataFrame(list(r.items()), columns= ['Feature','Mean Ranking'])

# Sort the dataframe
meanplot = meanplot.sort_values('Mean Ranking', ascending=False)

In [ ]:
# Let's plot the ranking of the features
sns.factorplot(x="Mean Ranking", y="Feature", data = meanplot, kind="bar", size=4, aspect=1.9, palette='cubehelix_r')

## linear regression

In [ ]:
X = df.drop('price',axis=1)
y = df['price']

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 101)
print("train:{}{}\n rest:{}{}".format(X_train.shape,y_train.shape,X_test.shape, y_test.shape))

In [ ]:
# stats OLS
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

In [ ]:
from sklearn import metrics

In [ ]:
def evaluation(true, predicted, train=True):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    if train:
        print("========Training Result=======")
        print('MAE: ', mae)
        print('MSE: ', mse)
        print('RMSE: ', rmse)
        print('R2 Square: ', r2_square)
    elif not train:
        print("=========Testing Result=======")
        print('MAE: ', mae)
        print('MSE: ', mse)
        print('RMSE: ', rmse)
        print('R2 Square: ', r2_square)

In [ ]:
evaluation(y_test, results.predict(X_test),train = False)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)


print("accuracy", lin_reg.score(X_test, y_test))
evaluation(y_train, lin_reg.predict(X_train),train = True)
evaluation(y_test, lin_reg.predict(X_test),train = False)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
linreg = LinearRegression()
def linReg(X, y):  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    linreg.fit(X_train, y_train)
    y_hat = linreg.predict(X_test)
    y_hat_train = linreg.predict(X_train)
    print('R_squared Score:', linreg.score(X, y))
    #Display errors
    print('Mean Absolute Error:', mean_absolute_error(y_test, y_hat))
    print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, y_hat)))
    print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
    #Compare predicted and actual values
    print('Mean Predicted Selling Price:', y_hat.mean())
    print('Mean Selling Price:', y_test.mean())
    score = cross_val_score(linreg, X, y,cv=10, scoring="r2")
    print('R_squared Mean Score:',score.mean())
    print(score)
    return linreg
linReg(X,y)

### LR with RFE

In [ ]:
def get_models():
	models = dict()
	for i in range(2, 18):
		rfe = RFE(estimator=LinearRegression(), n_features_to_select=i)
		model = LinearRegression()
		models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
	return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='r2', cv=10, n_jobs=-1, error_score='raise')
	return scores
 

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)

## Polynomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=2)
Xp= polynomial_features.fit_transform(X_train)
Xpr= polynomial_features.fit_transform(X)
Xpt =  polynomial_features.fit_transform(X_test)
import statsmodels.api as sm

model_poly = sm.OLS(y_train, Xp).fit()
#print(model_poly.summary())

print("R2: ", model_poly.rsquared)
evaluation(y_test, model_poly.predict(Xpt),train = False)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
lin_reg2 = LinearRegression()
lin_reg2.fit(Xp,y_train)
yhat = lin_reg2.predict(Xpt)
y_hat_train = lin_reg2.predict(Xp)
print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
print('Mean Predicted Selling Price:', yhat.mean())
print('Mean Selling Price:', y_test.mean())
score = cross_val_score(lin_reg2, Xpr, y,cv=10, scoring="r2")
print('R_squared Mean Score:',score.mean())
print(score)

## Ridge

In [ ]:
from sklearn.linear_model import Ridge
alpha = [0.01, 0.1, 0.5, 1, 1.5, 2, 2.5, 3, 5, 8, 10, 15]
r2=[]
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
for i in range(len(alpha)):
  RidgeModel = Ridge(alpha = alpha[i])
  RidgeModel.fit(X_train, y_train)
  yhat = RidgeModel.predict(X_test)
  y_hat_train = RidgeModel.predict(X_train)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(RidgeModel.score(X_test, y_test))
  #print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
  mae.append(mean_absolute_error(y_test, yhat))
  #print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
  rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
  #print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
  #print('Mean Predicted Selling Price:', yhat.mean())
  #print('Mean Selling Price:', y_test.mean())
  yhat_mean.append(yhat.mean())
  ytest_mean.append(y_test.mean())
  score = cross_val_score(RidgeModel, X, y,cv=10, scoring="r2")
  #print('R_squared Mean Score:',score.mean())
  r2_mean.append(score.mean())
  # print(score)

In [ ]:
r2_mean

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate ridge regression $R^2$ with different alphas')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, rmse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('RMSE')
plt.title(r'Evaluate ridge regression RMSE with different alphas')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, mae, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('MAE')
plt.title(r'Evaluate ridge regression MAE with different alphas')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(alpha, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('Price')
plt.title(r'Evaluate ridge regression Prices ith different alphas')
plt.legend(loc='best')
plt.grid()

plt.show()

## ridge with poly degree = 2

In [ ]:
pr = PolynomialFeatures(degree = 2)
x_train_pr = pr.fit_transform(X_train)
x_test_pr = pr.fit_transform(X_test)

mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
for i in range(len(alpha)):
  RidgeModel = Ridge(alpha = alpha[i])
  RidgeModel.fit(x_train_pr, y_train)
  yhat = RidgeModel.predict(x_test_pr)
  y_hat_train = RidgeModel.predict(x_train_pr)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(RidgeModel.score(x_test_pr, y_test))
  #print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
  mae.append(mean_absolute_error(y_test, yhat))
  #print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
  rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
  #print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
  #print('Mean Predicted Selling Price:', yhat.mean())
  #print('Mean Selling Price:', y_test.mean())
  yhat_mean.append(yhat.mean())
  ytest_mean.append(y_test.mean())
  score = cross_val_score(RidgeModel, x_pr, y,cv=10, scoring="r2")
  #print('R_squared Mean Score:',score.mean())
  r2_mean.append(score.mean())
  # print(score)

In [ ]:
r2_mean

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate ridge regression $R^2$ with different alphas')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, rmse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('RMSE')
plt.title(r'Evaluate ridge regression RMSE with different alphas')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, mae, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('MAE')
plt.title(r'Evaluate ridge regression MAE with different alphas')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(alpha, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('Price')
plt.title(r'Evaluate ridge regression Prices ith different alphas')
plt.legend(loc='best')
plt.grid()

plt.show()

In [ ]:
alpha = [0.01, 0.1, 0.5, 1, 1.5, 2, 2.5, 3, 5, 8, 10, 15]
pr = PolynomialFeatures(degree = 3)
x_train_pr = pr.fit_transform(X_train)
x_test_pr = pr.fit_transform(X_test)

r2 = []
for i in range(len(alpha)):
  RidgeModel = Ridge(alpha = alpha[i])
  RidgeModel.fit(x_train_pr, y_train)
  yhat = RidgeModel.predict(x_test_pr)
  y_hat_train = RidgeModel.predict(x_train_pr)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(RidgeModel.score(x_test_pr, y_test))

In [ ]:
r2

## lasso

In [ ]:
def Lasso_Regression(x_test, y_test, x_train, y_train, min_el, max_el):
    
    # Create a vector of lambdas
    ells = np.linspace(min_el, max_el, 50)
    num_lambdas = len(ells)
    num_predictors = np.shape(x_train)[1]
    
    # Empty arrays to store r2 values and coefficients
    train_r_squared = np.zeros(num_lambdas)
    test_r_squared = np.zeros(num_lambdas)
    coeff_a = np.zeros((num_lambdas, num_predictors))
    test_mse = np.zeros(num_lambdas)
    r2_mean = []
    mae = []
    rmse = []
    yhat_mean = []
    ytest_mean = []

    # Loop
    for i, ell in enumerate(ells):
        
        # Ridge regression
        reg = Lasso_Reg(alpha=ell)
        reg.fit(x_train, y_train)
       # val_mse[i] = np.sum((y_val-reg.predict(x_val))**2)/float(len(y_val))
        test_mse[i] = np.sum((y_test-reg.predict(x_test))**2)/float(len(y_test))
        yhat = reg.predict(x_test)
        y_hat_train = reg.predict(x_train)

        # Calculate R2
        r2_test = reg.score(x_test, y_test)
        #r2_val = reg.score(x_val, y_val)
        r2_train = reg.score(x_train, y_train)
        test_r_squared[i] = r2_test
        train_r_squared[i] = r2_train
        
        mae.append(mean_absolute_error(y_test, yhat))
        rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
        yhat_mean.append(yhat.mean())
        ytest_mean.append(y_test.mean())
        #val_r_squared[i] = r2_val
        coeff_a[i,:] = reg.coef_

        evaluation(y_train, reg.predict(X_train),train = True)
       # evaluation(y_val, reg.predict(X_val),train = False)
        evaluation(y_test, reg.predict(X_test),train = False)

        score = cross_val_score(reg, X, y,cv=10, scoring="r2")
        #print('R_squared Mean Score:',score.mean())
        r2_mean.append(score.mean())


        
    return train_r_squared, test_r_squared, yhat_mean, ytest_mean, mae, rmse, ells, test_mse, r2_mean

In [ ]:
train_r_squared, test_r_squared, yhat_mean, ytest_mean, mae, rmse, lambdas, test_mse, r2_mean =  Lasso_Regression(X_test, y_test, X_train, y_train, -5,15) 

In [ ]:
r2_mean

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(lambdas, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate lasso regression $R^2$ with different lambdas')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(lambdas, test_mse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('RMSE')
plt.title(r'Evaluate lasso regression RMSE with different alphas')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(lambdas, train_r_squared, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('MAE')
plt.title(r'Evaluate lasso regression MAE with different alphas')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(lambdas, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=1)
plt.plot(lambdas, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=1)
plt.xlabel('Alpha value'); plt.ylabel('Price')
plt.title(r'Evaluate lasso regression Prices ith different alphas')
plt.legend(loc='best')
plt.grid()

plt.show()

## KNN 

In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
r2 = []
n_neighbors = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i in range(len(n_neighbors)):
  knn = KNeighborsRegressor(n_neighbors=n_neighbors[i])
  knn.fit(X_train, y_train)
  yhat = knn.predict(X_test)
  y_hat_train = knn.predict(X_train)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(knn.score(X_test, y_test))
  #print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
  mae.append(mean_absolute_error(y_test, yhat))
  #print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
  rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
  #print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
  #print('Mean Predicted Selling Price:', yhat.mean())
  #print('Mean Selling Price:', y_test.mean())
  yhat_mean.append(yhat.mean())
  ytest_mean.append(y_test.mean())
  score = cross_val_score(knn, X, y,cv=10, scoring="r2")
  #print('R_squared Mean Score:',score.mean())
  r2_mean.append(score.mean())
  # print(score)

In [ ]:
r2_mean

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Number of Neighbors'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate KNN regressor $R^2$ with different number of neighbors')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, rmse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Number of Neighbors'); plt.ylabel('RMSE')
plt.title(r'Evaluate KNN regressor RMSE with different number of neighbors')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, mae, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('number of neighbors'); plt.ylabel('MAE')
plt.title(r'Evaluate KNN regressor MAE with different number of neighbors')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=1)
plt.plot(n_neighbors, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=1)
plt.xlabel('number of neighbors'); plt.ylabel('Price')
plt.title(r'Evaluate kNN regressor Prices with different number of neighbors')
plt.legend(loc='best')
plt.grid()

plt.show()

## Decision Tree

In [ ]:
parameters = {
    'max_depth': range (5, 20),
    'min_samples_leaf': [5, 10, 15, 20, 25]}

grid_search = GridSearchCV(DecisionTreeRegressor(), parameters, cv=10, return_train_score=False, scoring='neg_mean_squared_error', n_jobs=4)
grid_search.fit(X_train, y_train)
grid_results = pd.DataFrame(grid_search.cv_results_)
grid_search.best_estimator_

In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
r2 = []
model = grid_search.best_estimator_
yhat = model.predict(X_test)
y_hat_train = model.predict(X_train)
r2.append(model.score(X_test, y_test))
mae.append(mean_absolute_error(y_test, yhat))
rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
print('Mean Predicted Selling Price:', yhat.mean())
print('Mean Selling Price:', y_test.mean())
yhat_mean.append(yhat.mean())
ytest_mean.append(y_test.mean())
score = cross_val_score(model, X, y,cv=10, scoring="r2")
print('R_squared Mean Score:',score.mean())
r2_mean.append(score.mean())
print(score)


In [ ]:
grid_results.sort_values(by='rank_test_score').head(5)

In [ ]:
df_scores = grid_results.sort_values(by='rank_test_score')[['params', 'mean_test_score', 'std_test_score' ]]
df_scores[['mean_test_score',  'std_test_score']] = np.sqrt(df_scores[['mean_test_score',  'std_test_score']].abs())
df_scores.head()

## Random forest

In [ ]:
parameters = {
    'max_depth': range (5, 20),
    'n_estimators': [5, 10, 15, 20, 30]}

grid_search = GridSearchCV(RandomForestRegressor(), parameters, cv=10, return_train_score=False, scoring='neg_mean_squared_error', n_jobs=4)
grid_search.fit(X_train, y_train)
grid_results = pd.DataFrame(grid_search.cv_results_)
grid_search.best_estimator_


In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
r2 = []
model = grid_search.best_estimator_
yhat = model.predict(X_test)
y_hat_train = model.predict(X_train)
r2.append(model.score(X_test, y_test))
mae.append(mean_absolute_error(y_test, yhat))
rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
print('Mean Predicted Selling Price:', yhat.mean())
print('Mean Selling Price:', y_test.mean())
yhat_mean.append(yhat.mean())
ytest_mean.append(y_test.mean())
score = cross_val_score(model, X, y,cv=10, scoring="r2")
print('R_squared Mean Score:',score.mean())
r2_mean.append(score.mean())
print(score)


## XGBRegressor

In [ ]:
import xgboost as xgb
param={
    'objective': 'reg:linear',
    'eta'      :0.02,
    'eval_metric':'rmse',
    'max_depth': 5,
    'min_child_weight':3,
    'subsample' : 0.8,
    'colsample_bytree' : 0.8,
    'silent': 1,
    'seed' : 123
}
trn = xgb.DMatrix(X_train,label=y_train)
tst = xgb.DMatrix(X_test,label=y_test)
res = xgb.cv(param,trn,nfold=4,num_boost_round=2000,early_stopping_rounds=50,
             stratified=True,show_stdv=True,metrics={'rmse'},maximize=False)
min_index=np.argmin(res['test-rmse-mean'])

model = xgb.train(param,trn,min_index,[(trn,'train'),(tst,'test')])
pred = model.predict(tst)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test,pred)))

In [ ]:
r_sq = ((pred-np.mean(y_test))**2).sum() / ((y_test - np.mean(y_test))**2).sum()
print('R square is: ', r_sq)

## personal pipeline

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone # Clone estimator
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
pipelines = []
seed = 2

pipelines.append(
                ("Scaled_Ridge", 
                 Pipeline([
                     ("Scaler", StandardScaler()), 
                     ("Ridge", Ridge(random_state=seed, alpha=1 ))
                      ]))
                )
pipelines.append(
                ("Scaled_Lasso", 
                 Pipeline([
                     ("Scaler", StandardScaler()), 
                     ("Lasso", Lasso(random_state=seed, tol=1))
                      ]))
                )
pipelines.append(
                ("Scaled_Elastic", 
                 Pipeline([
                     ("Scaler", StandardScaler()), 
                     ("Lasso", ElasticNet(random_state=seed))
                      ]))
                )

pipelines.append(
                ("Scaled_SVR",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("SVR",  SVR(kernel='linear', C=1e2, degree=5))
                 ])
                )
                )

pipelines.append(
                ("Scaled_RF_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("RF", RandomForestRegressor(random_state=seed))
                 ])
                )
                )

pipelines.append(
                ("Scaled_ET_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("ET", ExtraTreesRegressor(random_state=seed))
                 ])
                )
                )
pipelines.append(
                ("Scaled_BR_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("BR", BaggingRegressor(random_state=seed))
                 ]))) 

pipelines.append(
                ("Scaled_Hub-Reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("Hub-Reg", HuberRegressor())
                 ]))) 
pipelines.append(
                ("Scaled_BayRidge",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("BR", BayesianRidge())
                 ]))) 

pipelines.append(
                ("Scaled_XGB_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("XGBR", XGBRegressor(seed=seed))
                 ]))) 

pipelines.append(
                ("Scaled_DT_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("DT_reg", DecisionTreeRegressor())
                 ]))) 

pipelines.append(
                ("Scaled_KNN_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("KNN_reg", KNeighborsRegressor())
                 ])))


pipelines.append(
                ("Scaled_Gboost-Reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("GBoost-Reg", GradientBoostingRegressor())
                 ])))

pipelines.append(
                ("Scaled_RFR_PCA",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("PCA", PCA(n_components=3)),
                     ("XGB", RandomForestRegressor())
                 ])))

pipelines.append(
                ("Scaled_XGBR_PCA",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("PCA", PCA(n_components=3)),
                     ("XGB", XGBRegressor())
                 ])))

scoring = 'r2'
n_folds = 10

results, names  = [], [] 

for name, model  in pipelines:
    kfold = KFold(n_splits=n_folds)
    cv_results = cross_val_score(model, X, y, cv= kfold,
                                 scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(),  cv_results.std())
    print(msg)

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(15,6))
fig.suptitle('Algorithm Comparison', fontsize=22)
ax = fig.add_subplot(111)
pyplot.boxplot(results, labels=names, showmeans=True)
ax.set_xticklabels(names)
ax.set_xlabel("Algorithmn Name", fontsize=20)
ax.set_ylabel("R Squared Score of Models", fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.show()


## DNN

In [ ]:
train_dataset = df.sample(frac=0.8,random_state=0)
test_dataset = df.drop(train_dataset.index)

train_stats = train_dataset.describe()
train_stats.pop('price')
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('price')
test_labels = test_dataset.pop('price')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
from sklearn.metrics import explained_variance_score

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(21, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(21, activation='relu'),
    layers.Dense(21, activation='relu'),
    layers.Dense(1),
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
predictions=model.predict(normed_test_data)
print("The Variance Score :", explained_variance_score(test_labels, predictions))

In [ ]:
mean_absolute_error(test_labels,predictions)

In [ ]:
mean_squared_error(test_labels,predictions)**0.5

In [ ]:
r2_score(test_labels,predictions)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(21, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(21, activation='relu'),
    layers.Dense(21, activation='relu'),
    layers.Dense(1),
  ])

  optimizer = tf.keras.optimizers.Adam()

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
predictions=model.predict(normed_test_data)
print("The Variance Score :", explained_variance_score(test_labels, predictions))

In [ ]:
mean_squared_error(test_labels,predictions)**0.5

In [ ]:
mean_absolute_error(test_labels,predictions)

In [ ]:
r2_score(test_labels,predictions)

# predicting log price

In [ ]:
X = data.drop(['id','price','log_price','sqft_living15','sqft_lot15','sqft_living','sqft_lot','sqft_above',
              'sqft_basement','zipcode'],axis=1)
y = data['log_price']

## ols

In [ ]:
# simple linear regression to explore features we think are interesting

#Define target 'y'(dependendent variable) and features 'X'(independent variables or predictors).
import statsmodels.formula.api as smf

#Test each continus variable as a standalone linear regression.
col_names = X.select_dtypes('float').columns
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Selling price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = smf.ols(formula=f, data=data).fit()
    X_new = pd.DataFrame({val: [X[val].min(), X[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[idx+1])

In [ ]:
#Define target 'y'(dependendent variable) and features 'X'(independent variables or predictors).
import statsmodels.formula.api as smf
#Test each continus variable as a standalone linear regression.
col_names = X.select_dtypes('int').columns
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Selling price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = smf.ols(formula=f, data=data).fit()
    X_new = pd.DataFrame({val: [X[val].min(), X[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[idx+1])

## RFE

In [ ]:
ranks = {}
colnames = X.columns
# Using Linear Regression
lr = LinearRegression(normalize=True)
lr.fit(X,y)
ranks["LinReg"] = ranking(np.abs(lr.coef_), colnames)

# Using Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X,y)
ranks['Ridge'] = ranking(np.abs(ridge.coef_), colnames)

# Using Lasso
lasso = Lasso(alpha=.05)
lasso.fit(X, y)
ranks["Lasso"] = ranking(np.abs(lasso.coef_), colnames)

rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=3)
rf.fit(X,y)
ranks["RF"] = ranking(rf.feature_importances_, colnames)

r = {}
for name in colnames:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
 
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("\t%s" % "\t".join(methods))
for name in colnames:
    print("%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods]))))

In [ ]:
meanplot = pd.DataFrame(list(r.items()), columns= ['Feature','Mean Ranking'])

# Sort the dataframe
meanplot = meanplot.sort_values('Mean Ranking', ascending=False)

In [ ]:
# Let's plot the ranking of the features
sns.factorplot(x="Mean Ranking", y="Feature", data = meanplot, kind="bar", size=4, aspect=1.9, palette='cubehelix_r')

## linear regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 111)

In [ ]:
# stats OLS
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

In [ ]:
def evaluation(true, predicted, train=True):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    if train:
        print("========Training Result=======")
        print('MAE: ', mae)
        print('MSE: ', mse)
        print('RMSE: ', rmse)
        print('R2 Square: ', r2_square)
    elif not train:
        print("=========Testing Result=======")
        print('MAE: ', mae)
        print('MSE: ', mse)
        print('RMSE: ', rmse)
        print('R2 Square: ', r2_square)

In [ ]:
evaluation(y_train, results.predict(X_train),train = True)
evaluation(y_test, results.predict(X_test),train = False)

In [ ]:
linreg = LinearRegression()
def linReg(X, y):  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    linreg.fit(X_train, y_train)
    y_hat = linreg.predict(X_test)
    y_hat_train = linreg.predict(X_train)
    print('R_squared Score:', linreg.score(X, y))
    #Display errors
    print('Mean Absolute Error:', mean_absolute_error(y_test, y_hat))
    print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, y_hat)))
    print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
    #Compare predicted and actual values
    print('Mean Predicted Selling Price:', y_hat.mean())
    print('Mean Selling Price:', y_test.mean())
    score = cross_val_score(linreg, X, y,cv=10, scoring="r2")
    print('R_squared Mean Score:',score.mean())
    explained = cross_val_score(linreg, X, y,cv=10, scoring="explained_variance")
    print('Explained variance:',explained.mean())
    print(score)
    return linreg
linReg(X,y)

In [ ]:
def get_models():
	models = dict()
	for i in range(2, 18):
		rfe = RFE(estimator=LinearRegression(), n_features_to_select=i)
		model = LinearRegression()
		models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
	return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='r2', cv=10, n_jobs=-1, error_score='raise')
	return scores
 

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)

## polynomial

In [ ]:
polyfeat=PolynomialFeatures(degree=2)
xtrain_poly=polyfeat.fit_transform(X_train)
xtest_poly=polyfeat.fit_transform(X_test)

poly=LinearRegression()
poly.fit(xtrain_poly,y_train)
polypred=poly.predict(xtest_poly)

print('Complex Model_3')
mean_squared_error = metrics.mean_squared_error(y_test, polypred)
print('Mean Squared Error (MSE) ', round(np.sqrt(mean_squared_error), 2))
print('R-squared (training) ', round(poly.score(xtrain_poly, y_train), 3))
print('R-squared (testing) ', round(poly.score(xtest_poly, y_test), 3))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=3)
Xp= polynomial_features.fit_transform(X_train)
Xpr= polynomial_features.fit_transform(X)
Xpt =  polynomial_features.fit_transform(X_test)
import statsmodels.api as sm

model_poly = sm.OLS(y_train, Xp).fit()
#print(model_poly.summary())

print("R2: ", model_poly.rsquared)
evaluation(y_test, model_poly.predict(Xpt),train = False)

## ridge

In [ ]:
from sklearn.linear_model import Ridge
alpha = [0.01, 0.1, 0.5, 1, 1.5, 2, 2.5, 3, 5, 8, 10, 15]
r2=[]
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
explained_var = []
for i in range(len(alpha)):
  RidgeModel = Ridge(alpha = alpha[i])
  RidgeModel.fit(X_train, y_train)
  yhat = RidgeModel.predict(X_test)
  y_hat_train = RidgeModel.predict(X_train)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(RidgeModel.score(X_test, y_test))
  #print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
  mae.append(mean_absolute_error(y_test, yhat))
  #print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
  rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
  #print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
  #print('Mean Predicted Selling Price:', yhat.mean())
  #print('Mean Selling Price:', y_test.mean())
  yhat_mean.append(yhat.mean())
  ytest_mean.append(y_test.mean())
  score = cross_val_score(RidgeModel, X, y,cv=10, scoring="r2")
  #print('R_squared Mean Score:',score.mean())
  explained = cross_val_score(RidgeModel, X, y,cv=10, scoring="explained_variance")
  #print('R_squared Mean Score:',explained.mean())
  r2_mean.append(score.mean())
  explained_var.append(explained.mean())
  # print(score)

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(alpha, explained_var, 'bo-', label=r'mean explained variance of 10-fold CV', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate ridge regression $R^2$ with different alphas')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, rmse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('RMSE')
plt.title(r'Evaluate ridge regression RMSE with different alphas')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, mae, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('MAE')
plt.title(r'Evaluate ridge regression MAE with different alphas')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(alpha, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('Price')
plt.title(r'Evaluate ridge regression Prices ith different alphas')
plt.legend(loc='best')
plt.grid()

plt.show()

## ridge + poly

In [ ]:
pr = PolynomialFeatures(degree = 2)
x_train_pr = pr.fit_transform(X_train)
x_test_pr = pr.fit_transform(X_test)

RidgeModel = Ridge(alpha = 0.1)
RidgeModel.fit(x_train_pr, y_train)
yhat = RidgeModel.predict(x_test_pr)
y_hat_train = RidgeModel.predict(x_train_pr)

print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))

In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
explained_var = []
for i in range(len(alpha)):
  RidgeModel = Ridge(alpha = alpha[i])
  RidgeModel.fit(x_train_pr, y_train)
  yhat = RidgeModel.predict(x_test_pr)
  y_hat_train = RidgeModel.predict(x_train_pr)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(RidgeModel.score(x_test_pr, y_test))
  #print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
  mae.append(mean_absolute_error(y_test, yhat))
  #print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
  rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
  #print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
  #print('Mean Predicted Selling Price:', yhat.mean())
  #print('Mean Selling Price:', y_test.mean())
  yhat_mean.append(yhat.mean())
  ytest_mean.append(y_test.mean())
  score = cross_val_score(RidgeModel, x_pr, y,cv=10, scoring="r2")
  #print('R_squared Mean Score:',score.mean())
  explained = cross_val_score(RidgeModel, x_pr, y , cv= 10, scoring = "explained_variance")
  r2_mean.append(score.mean())
  explained_var.append(explained.mean())
  # print(score)

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(alpha, explained_var, 'bo-', label='mean explained variance of 10-fold CV', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate ridge regression $R^2$ with different alphas')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, rmse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('RMSE')
plt.title(r'Evaluate ridge regression RMSE with different alphas')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, mae, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=3)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('MAE')
plt.title(r'Evaluate ridge regression MAE with different alphas')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(alpha, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(alpha, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Alpha value'); plt.ylabel('Price')
plt.title(r'Evaluate ridge regression Prices ith different alphas')
plt.legend(loc='best')
plt.grid()

plt.show()

In [ ]:
r2_mean

## lasso

In [ ]:
def Lasso_Regression(x_test, y_test, x_train, y_train, min_el, max_el):
    
    # Create a vector of lambdas
    ells = np.linspace(min_el, max_el, 50)
    num_lambdas = len(ells)
    num_predictors = np.shape(x_train)[1]
    
    # Empty arrays to store r2 values and coefficients
    train_r_squared = np.zeros(num_lambdas)
    test_r_squared = np.zeros(num_lambdas)
    coeff_a = np.zeros((num_lambdas, num_predictors))
    test_mse = np.zeros(num_lambdas)
    r2_mean = []
    mae = []
    rmse = []
    yhat_mean = []
    ytest_mean = []
    explained_var = []

    # Loop
    for i, ell in enumerate(ells):
        
        # Ridge regression
        reg = Lasso_Reg(alpha=ell)
        reg.fit(x_train, y_train)
       # val_mse[i] = np.sum((y_val-reg.predict(x_val))**2)/float(len(y_val))
        test_mse[i] = np.sum((y_test-reg.predict(x_test))**2)/float(len(y_test))
        yhat = reg.predict(x_test)
        y_hat_train = reg.predict(x_train)

        # Calculate R2
        r2_test = reg.score(x_test, y_test)
        #r2_val = reg.score(x_val, y_val)
        r2_train = reg.score(x_train, y_train)
        test_r_squared[i] = r2_test
        train_r_squared[i] = r2_train
        
        mae.append(mean_absolute_error(y_test, yhat))
        rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
        yhat_mean.append(yhat.mean())
        ytest_mean.append(y_test.mean())
        #val_r_squared[i] = r2_val
        coeff_a[i,:] = reg.coef_

        #evaluation(y_train, reg.predict(X_train),train = True)
       # evaluation(y_val, reg.predict(X_val),train = False)
        #evaluation(y_test, reg.predict(X_test),train = False)

        score = cross_val_score(reg, X, y,cv=10, scoring="r2")
        #print('R_squared Mean Score:',score.mean())
        r2_mean.append(score.mean())

        explained =  cross_val_score(reg, X, y,cv=10, scoring="explained_variance")
        explained_var.append(explained.mean())
        
    return train_r_squared, test_r_squared, yhat_mean, ytest_mean, mae, rmse, ells, test_mse, r2_mean,explained_var

In [ ]:
train_r_squared, test_r_squared, yhat_mean, ytest_mean, mae, rmse, lambdas, test_mse, r2_mean, explained_var=  Lasso_Regression(X_test, y_test, X_train, y_train, -5,15) 

## KNN 

In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
r2 = []
explained_var = []
n_neighbors = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i in range(len(n_neighbors)):
  knn = KNeighborsRegressor(n_neighbors=n_neighbors[i])
  knn.fit(X_train, y_train)
  yhat = knn.predict(X_test)
  y_hat_train = knn.predict(X_train)
  #print("The R^2 Score value for the training data is : " + str(RidgeModel.score(x_train_pr, y_train)))
  #print("The R^2 Score value for the testing data is : " + str(RidgeModel.score(x_test_pr, y_test)))
  r2.append(knn.score(X_test, y_test))
  #print('Mean Absolute Error:', mean_absolute_error(y_test, yhat))
  mae.append(mean_absolute_error(y_test, yhat))
  #print('Root Mean Squared Error test:', np.sqrt(mean_squared_error(y_test, yhat)))
  rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
  #print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
  #print('Mean Predicted Selling Price:', yhat.mean())
  #print('Mean Selling Price:', y_test.mean())
  yhat_mean.append(yhat.mean())
  ytest_mean.append(y_test.mean())
  score = cross_val_score(knn, X, y,cv=10, scoring="r2")
  var = cross_val_score(knn, X, y,cv=10, scoring="explained_variance")
  #print('R_squared Mean Score:',score.mean())
  r2_mean.append(score.mean())
  explained_var.append(var.mean())
  # print(score)

In [ ]:
r2_mean

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 2, 1)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, r2_mean, 'bo-', label=r'mean $R^2$ of 10-fold CV', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Number of Neighbors'); plt.ylabel(r'$R^2$')
plt.title(r'Evaluate KNN regressor $R^2$ with different number of neighbors')
plt.legend(loc='best')
plt.grid()


plt.subplot(2, 2, 2)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, rmse, 'bo-', label='RMSE of testing set', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Number of Neighbors'); plt.ylabel('RMSE')
plt.title(r'Evaluate KNN regressor RMSE with different number of neighbors')
plt.legend(loc='best')
plt.grid()



plt.subplot(2, 2, 3)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, mae, 'bo-', label='MAE of testing set', color="darkblue", alpha=0.6, linewidth=1)
#plt.plot(val_r_squared, 'bo-', label=r'$R^2$ Val set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('number of neighbors'); plt.ylabel('MAE')
plt.title(r'Evaluate KNN regressor MAE with different number of neighbors')
plt.legend(loc='best')
plt.grid()

plt.subplot(2, 2, 4)
# Plotting
#plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, yhat_mean, 'bo-', label='prediction of testing set', color="darkblue", alpha=0.6, linewidth=1)
plt.plot(n_neighbors, ytest_mean, 'bo-', label='true value of testing set', color="darkred", alpha=0.6, linewidth=1)
plt.xlabel('number of neighbors'); plt.ylabel('Price')
plt.title(r'Evaluate kNN regressor Prices with different number of neighbors')
plt.legend(loc='best')
plt.grid()

plt.show()

## decision tree

In [ ]:
parameters = {
    'max_depth': range (5, 20),
    'min_samples_leaf': [5, 10, 15, 20, 25]}

grid_search = GridSearchCV(DecisionTreeRegressor(), parameters, cv=10, return_train_score=False, scoring='neg_mean_squared_error', n_jobs=4)
grid_search.fit(X_train, y_train)
grid_results = pd.DataFrame(grid_search.cv_results_)
grid_search.best_estimator_

In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
r2 = []
explained_var =[]
cv_rmse = []
model = DecisionTreeRegressor(max_depth=12, min_samples_leaf=15)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
y_hat_train = model.predict(X_train)
r2.append(model.score(X_test, y_test))
mae.append(mean_absolute_error(y_test, yhat))
rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
print('Mean Predicted Selling Price:', yhat.mean())
print('Mean Selling Price:', y_test.mean())
yhat_mean.append(yhat.mean())
ytest_mean.append(y_test.mean())
score = cross_val_score(model, X, y,cv=10, scoring="r2")
print('R_squared Mean Score:',score.mean())
var = cross_val_score(model, X, y,cv=10, scoring="explained_variance")
print('explained var Mean Score:',var.mean())
cv_rmse = cross_val_score(model, X, y,cv=10, scoring="neg_root_mean_squared_error")
print('RMSE Mean Score:',cv_rmse.mean())
r2_mean.append(score.mean())
print(score)
print(var)

In [ ]:
df_scores = grid_results.sort_values(by='rank_test_score')[['params', 'mean_test_score', 'std_test_score' ]]
df_scores[['mean_test_score',  'std_test_score']] = np.sqrt(df_scores[['mean_test_score',  'std_test_score']].abs())
df_scores.head()

## random forest

In [ ]:
parameters = {
    'max_depth': range (5, 20),
    'n_estimators': [5, 10, 15, 20, 30]}

grid_search = GridSearchCV(RandomForestRegressor(), parameters, cv=10, return_train_score=False, scoring='neg_mean_squared_error', n_jobs=4)
grid_search.fit(X_train, y_train)
grid_results = pd.DataFrame(grid_search.cv_results_)
grid_search.best_estimator_

In [ ]:
mae = []
rmse = []
r2_mean = []
yhat_mean = []
ytest_mean = []
r2 = []
model = RandomForestRegressor(max_depth=16,
                              n_estimators=30)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
y_hat_train = model.predict(X_train)
r2.append(model.score(X_test, y_test))
mae.append(mean_absolute_error(y_test, yhat))
rmse.append(np.sqrt(mean_squared_error(y_test, yhat)))
print('Root Mean Squared Error train:', np.sqrt(mean_squared_error(y_train, y_hat_train)))
print('Mean Predicted Selling Price:', yhat.mean())
print('Mean Selling Price:', y_test.mean())
yhat_mean.append(yhat.mean())
ytest_mean.append(y_test.mean())
score = cross_val_score(model, X, y,cv=10, scoring="r2")
print('R_squared Mean Score:',score.mean())
var = cross_val_score(model, X, y,cv=10, scoring="explained_variance")
print('explained var Mean Score:',var.mean())
cv_rmse = cross_val_score(model, X, y,cv=10, scoring="neg_root_mean_squared_error")
print('RMSE Mean Score:',cv_rmse.mean())
r2_mean.append(score.mean())
print(score)


## XGB

In [ ]:
import xgboost as xgb
param={
    'objective': 'reg:linear',
    'eta'      :0.02,
    'eval_metric':'rmse',
    'max_depth': 5,
    'min_child_weight':3,
    'subsample' : 0.8,
    'colsample_bytree' : 0.8,
    'silent': 1,
    'seed' : 123
}
trn = xgb.DMatrix(X_train,label=y_train)
tst = xgb.DMatrix(X_test,label=y_test)
res = xgb.cv(param,trn,nfold=4,num_boost_round=2000,early_stopping_rounds=50,
            show_stdv=True,metrics={'rmse'},maximize=False)
min_index=np.argmin(res['test-rmse-mean'])

model = xgb.train(param,trn,min_index,[(trn,'train'),(tst,'test')])
pred = model.predict(tst)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test,pred)))

In [ ]:
r_sq = ((pred-np.mean(y_test))**2).sum() / ((y_test - np.mean(y_test))**2).sum()
print('R square is: ', r_sq)

##  Personal pipeline

In [ ]:
pipelines = []
seed = 2

pipelines.append(
                ("Scaled_Ridge", 
                 Pipeline([
                     ("Scaler", StandardScaler()), 
                     ("Ridge", Ridge(random_state=seed, alpha=0.01 ))
                      ]))
                )
pipelines.append(
                ("Scaled_Lasso", 
                 Pipeline([
                     ("Scaler", StandardScaler()), 
                     ("Lasso", Lasso(random_state=seed, tol=1))
                      ]))
                )
pipelines.append(
                ("Scaled_Elastic", 
                 Pipeline([
                     ("Scaler", StandardScaler()), 
                     ("Lasso", ElasticNet(random_state=seed))
                      ]))
                )


pipelines.append(
                ("Scaled_RF_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("RF", RandomForestRegressor(max_depth=16, n_estimators=30,random_state = seed))
                 ])
                )
                )

pipelines.append(
                ("Scaled_ET_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("ET", ExtraTreesRegressor(random_state=seed))
                 ])
                )
                )
pipelines.append(
                ("Scaled_BR_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("BR", BaggingRegressor(random_state=seed))
                 ]))) 

pipelines.append(
                ("Scaled_Hub-Reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("Hub-Reg", HuberRegressor())
                 ]))) 
pipelines.append(
                ("Scaled_BayRidge",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("BR", BayesianRidge())
                 ]))) 

pipelines.append(
                ("Scaled_XGB_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("XGBR", XGBRegressor( ))])))
pipelines.append(
                ("Scaled_DT_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("DT_reg", DecisionTreeRegressor(max_depth = 12, min_samples_leaf = 15))
                 ]))) 

pipelines.append(
                ("Scaled_KNN_reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("KNN_reg", KNeighborsRegressor())
                 ])))
pipelines.append(
                ("Scaled_ADA-Reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("ADA-reg", AdaBoostRegressor())
                 ]))) 

pipelines.append(
                ("Scaled_Gboost-Reg",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("GBoost-Reg", GradientBoostingRegressor())
                 ])))

pipelines.append(
                ("Scaled_RFR_PCA",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("PCA", PCA(n_components=3)),
                     ("XGB", RandomForestRegressor())
                 ])))

pipelines.append(
                ("Scaled_XGBR_PCA",
                 Pipeline([
                     ("Scaler", StandardScaler()),
                     ("PCA", PCA(n_components=3)),
                     ("XGB", XGBRegressor())
                 ])))

#'neg_mean_absolute_error', 'neg_mean_squared_error','r2'
scoring = 'r2'
n_folds = 10

results, names  = [], [] 
var, rmse = [],[]

for name, model  in pipelines:
    kfold = KFold(n_splits=n_folds)
    cv_results = cross_val_score(model, X, y, cv= kfold,
                                 scoring=scoring, n_jobs=-1) 
    cv_r = cross_val_score(model, X, y, cv= kfold,
                                 scoring='explained_variance', n_jobs=-1) 
    cv_rmse =   cross_val_score(model, X, y, cv= kfold,
                                 scoring='neg_root_mean_squared_error', n_jobs=-1)
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(),  cv_results.std())
    print(msg)
    var.append(cv_r)
    msg_2 = "%s: %f (+/- %f)" % (name, cv_r.mean(),  cv_r.std())
    print(msg_2)
    rmse.append(cv_rmse)
    msg_3 = "%s: %f (+/- %f)" % (name, cv_rmse.mean(),  cv_rmse.std())
    print(msg_3)

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(15,6))
fig.suptitle('Algorithm Comparison', fontsize=22)
ax = fig.add_subplot(111)
rm = [ -x for x in rmse]
pyplot.boxplot(rm, labels=names, showmeans=True)
ax.set_xticklabels(names)
ax.set_xlabel("Algorithmn Name", fontsize=20)
ax.set_ylabel("RMSE Score of Models", fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,6))
fig.suptitle('Algorithm Comparison', fontsize=22)
ax = fig.add_subplot(111)
pyplot.boxplot(results, labels=names, showmeans=True)
ax.set_xticklabels(names)
ax.set_xlabel("Algorithmn Name", fontsize=20)
ax.set_ylabel("R squared Score of Models", fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,6))
fig.suptitle('Algorithm Comparison', fontsize=22)
ax = fig.add_subplot(111)
pyplot.boxplot(var, labels=names, showmeans=True)
ax.set_xticklabels(names)
ax.set_xlabel("Algorithmn Name", fontsize=20)
ax.set_ylabel("R squared Score of Models", fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.show()

## DNN

In [ ]:
df['log_price'] = np.log(df['price'])

In [ ]:
df =df.drop(columns=["price"])

In [ ]:
train_dataset = df.sample(frac=0.8,random_state=0)
test_dataset = df.drop(train_dataset.index)

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop('log_price')
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('log_price')
test_labels = test_dataset.pop('log_price')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
normed_train_data.shape

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(21, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(21, activation='relu'),
    layers.Dense(21, activation='relu'),
    layers.Dense(1),
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
predictions=model.predict(normed_test_data)
print("The Variance Score :", explained_variance_score(test_labels, predictions))

In [ ]:
mean_absolute_error(test_labels,predictions)

In [ ]:
mean_squared_error(test_labels,predictions)**0.5

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(21, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(21, activation='relu'),
    layers.Dense(21, activation='relu'),
    layers.Dense(1),
  ])

  optimizer = tf.keras.optimizers.Adam()

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
predictions=model.predict(normed_test_data)
print("The Variance Score :", explained_variance_score(test_labels, predictions))

In [ ]:
mean_absolute_error(test_labels,predictions)

In [ ]:
mean_squared_error(test_labels,predictions)**0.5

reference

data source:
https://www.kaggle.com/datasets/harlfoxem/housesalesprediction

https://www.kaggle.com/code/dktalaicha/house-price-prediction-with-keras#Evaluation-on-Test-Data

https://www.kaggle.com/code/arthurtok/feature-ranking-rfe-random-forest-linear-models

https://www.kaggle.com/code/vivianbenben/exploration-and-xgboost-to-predict-housing-price

https://www.kaggle.com/code/kabure/predicting-house-prices-xgb-rf-bagging-reg-pipe